In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!pip install -q datasets transformers accelerate demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset('csv', data_files='/content/drive/MyDrive/data.csv') #data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 93907
    })
})

In [ ]:
data = data.filter(lambda x: x['context_1'] is not None and x['response'] is not None)

Filter:   0%|          | 0/93907 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 57945
    })
})

In [ ]:
import re
import demoji

def preproc(row):
    for key in row:
        if row[key] is not None:
            row[key] = row[key].replace('\n',' ')
            row[key] = re.sub(' +', ' ', row[key])
            row[key] = demoji.replace(row[key], '')
    return row

In [ ]:
data = data.map(preproc)

Map:   0%|          | 0/57945 [00:00<?, ? examples/s]

In [ ]:
data = data['train'].train_test_split(test_size = 0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 46356
    })
    test: Dataset({
        features: ['context_3', 'context_2', 'context_1', 'response'],
        num_rows: 11589
    })
})

In [ ]:
def format(row):
    if 'context_3' in row:
        text = f"@@ПЕРВЫЙ@@ {row['context_3']} @@ВТОРОЙ@@ {row['context_2']} @@ПЕРВЫЙ@@ {row['context_1']} @@ВТОРОЙ@@ {row['response']}"
    elif 'context_2' in row:
        text = f"@@ПЕРВЫЙ@@ {row['context_2']} @@ВТОРОЙ@@ {row['context_1']} @@ПЕРВЫЙ@@ {row['response']}"
    else:
        text = f"@@ПЕРВЫЙ@@ {row['context_1']} @@ВТОРОЙ@@ {row['response']}"

    return {'text': text}


In [ ]:
data = data.map(format, remove_columns=['context_3', 'context_2', 'context_1', 'response'])

Map:   0%|          | 0/46356 [00:00<?, ? examples/s]

Map:   0%|          | 0/11589 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 46356
    })
    test: Dataset({
        features: ['text'],
        num_rows: 11589
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium').to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
data = data.map(lambda x: tokenizer(x['text'], max_length = 256, truncation=True, padding=True), remove_columns=['text'])

Map:   0%|          | 0/46356 [00:00<?, ? examples/s]

Map:   0%|          | 0/11589 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 46356
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11589
    })
})

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/sirius3',
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    save_steps =  500,
    eval_steps = 10,
    save_total_limit = 2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=data['train'],
    eval_dataset=data['test']
)

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.529200
1000,3.364100
1500,3.269900
2000,3.215800
2500,3.155000
3000,3.056500
3500,2.841900
4000,2.814100
4500,2.795500


Step,Training Loss
500,3.529200
1000,3.364100
1500,3.269900
2000,3.215800
2500,3.155000
3000,3.056500
3500,2.841900
4000,2.814100
4500,2.795500
5000,2.764800


In [ ]:
from transformers import AutoModelWithLMHead

checkpoint_path = '/content/drive/MyDrive/sirius2/checkpoint-1050'

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained(checkpoint_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
inputs = tokenizer('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ хай @@ПЕРВЫЙ@@ Что купить поесть? @@ВТОРОЙ@@ ', return_tensors='pt')
generated_token_ids = model.generate(**inputs)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ хай @@ПЕРВЫЙ@@ Что купить поесть? @@ВТОРОЙ@@ щами да хинкали с']